## ISIC-2018 比赛总结（Task1）

### 一、 问题阐述

本次参加的比赛是[**ISIC Skin Image Analysis Workshop and Challenge@_MICCAI 2018_**](https://challenge2018.isic-archive.com/)，我和飞飞做的是 [**Task 1: Lesion Boundary Segmentation**](https://challenge2018.isic-archive.com/task1/), 6月25日开始着手，7月27日截止提交。    
这是一个比较基本的二分类的语义分割问题。

![](./task1.png)

**特别之处：**   
1. 本次比赛是用的计分方式为jaccard得分，如果jaccard得分低于0.65分的话则算为0分，其表示如下：
``` python
jaccard = jaccard if jaccard > 0.65 else 0
```

    其中，一些常用的计分函数的公式总结如下在 [*scipy.spatial.distance*](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#module-scipy.spatial.distance) 这个包里,如在语义分割上常常用到的[*dice*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.dice.html#scipy.spatial.distance.dice), jaccard，hamming等，如果有需要的话可以回来使用，需要注意的是这个distance包计算计算的是两者差距（distance，距离）而不是相似行，对其的相似性进行评价的时候，需要做的是
``` python
similarity = 1 - distance
```

2. 另外，官方说明，每一张图像中官方只会提供一个病灶的标签数据，哪怕我们在观察数据的过程中发现有部分图像存在两个乃至多个明显的病灶，一般标注出来的是lesion区域是全图中最大的那一块。（这在后来导致了后处理过程的产生）
![](./ISIC_0007141.jpg)    
![](./ISIC_0007141_segmentation.png)   

### 二、 数据预处理过程

  在官方提供的数据集中，我们拿到的训练集有2659张带分割任务标签的图像，在临近比赛结束的时候（7月9号左右的样子），官网公布了验证集100张图像（只拿得到皮肤镜图像，拿不到标签数据，但是可以通过提交分割结果结果得分），并且公布了最终的测试机1000张皮肤镜图像，而测试集的分割标签是永不公开的。      

在实际使用的时候，我们能靠得住的就只有2659张训练集的图像，对此，按照 8:2 的比例随机分成了本生的训练集和测试集。

在数据预处理的过程中，遇到了几个问题，在这里列举和讨论一下（尽管有些问题还是比较的简单的）
1. 数据预处理的过程中，发现数据集中图像的大小差异很大，小的图像约为 400 × 500 左右，大的图像有 4000 × 6000 的样子，这种情况改怎么办？   
   1） 在进行语义分割的全卷积神经网络中，网络对输入图像的大小并没有严格的要求   
   2） 验证集的图像大小是不一致的，我们是否需要对验证集的图像进行与训练集图像同样的处理？
2. 去均值归一化的问题    
   1） 我最开始的进行的去均值归一化过程做的方式如下代码所示，请问这样子是否有问题？
   ``` python
   def Gaussion_normalization( img ):
       img = img.astype( float )
       for i in range( img.shape[2] ):
          channel   = img[:,:,i]
          mean      = np.mean( channel )
          std       = np.std( channel )
          img[,,i] -= mean
          img[,,i] /= std
       return img 
   ```
   2） 去均值和归一化的作用分别是什么？   
   3） 正确去均值和归一化的方法是怎么样的？   
   4） RGB图像应不应该进行去均值和归一化的过程？
   

### 三、 训练模型

整个实验的过程中，使用的基本的网络结构是 U-net 结构, 并且在一下方面进行了尝试：

- 参数的初始问题   
    之前做毕业设计的时候，我专门总结过参数初始化的一堆方法，包括非常随意的随机数初始化、为了预防深层网络的梯度消失和梯度爆炸的而要符合一定分布规律的Xavier初始化方法及其针对relu这种激活函数的改进型号MSRA（he）初始化方法。然而实践证明，以上都不是最好的初始化方法，最快达到 baseline 的初始化方法是将参数设置为 pretrained model 的参数，如果没有 pretrained model 的话，不妨讲网络前几层的参数设置为如 Vgg16、Vgg19一类预训练模型的参数。这样子可以更快地学习到数据的特征。当然，不是说初始化为随机数的性能就一定比不上pretrained model的，而是需要学习更多的次数才能找到类似的方法局部最优解。   
    在本次实验中使用了 Vgg16 来初始化 u-net 的向下卷基层的参数，同时也尝试了使用随机数来初始化全部参数的实验，在自身的验证集上查了8个点左右。
    同时，在使用pretrained Model 的时候要数据预处理时的去均值和归一化问题

- 加深网络的层数的问题   
    在开始的时候，u-net 的 downsample 的次数设置为 3，向下卷积的层数为8层，网络的卷积层数为15层，这时候，网络的分割结果比较的LOCAL，也就是网络的感受野比较小，面积较大的病变区域分割结果很差。设置 downsample 的次数为 4 的时候，向下卷积层的数目为10层，此时结果比较理想。设置 downsample 次数为 5 的时候， 向下卷积层的数目为 12 层，结果却变差了，其原因参考 resnet 产生的原因。（在模型结构比较简单的时候，粗暴地增大网络的层数并不会提高网络的性能，反而因为训练的困难而产生反作用）

- 输入输出的问题   
    标准的 U-net 的输出的结果比网络输入的图像要小很多， 这种小不是 scale 的小，而是 crop 出来的小， 因为作者默认了一般病变图像的病灶中心，而在我们的数据集中，一些的病灶区域都是占据了几乎整张的图像的，这种情况下，crop后的分割结果会严重影响图像边缘的分割精度。
    因此，我们将 U-net 中的卷积过程全部修改成了添加 0 Padding的方式，使得网络输入的图像与输出图像的大小保持一致。

- 修改网络结构   
    参考 denseNet 将 U-net 修改地更加的 dense， 这种情况下，网络变得不稳定了，而且最高点的精度也在下降。 

- 修改输入网络的图像的 size 大小
    size比较大的图像在病灶比较小的时候，保持这比较好的结果，但是在病灶很大的时候，会出现一些空洞的情况，不能很好地囊括整个病变区域； 而在 size 比较小的情况下，大区域的病变区域分割结果很好，小区域的病灶容易被网络忽略掉。

- 数据的augmentation
    自己的验证集不应该做 augmentation， augmentation之后的数据也不应该超过原来数据集的2倍。本次比赛中使用的验证方式有augmentation 有随机的上下翻转，左右翻转，随机旋转一个0到45度之间的角度。做了augmentation之后，结果没有明显的提升，但是模型变得更加稳定了


### 四、 结果分析

由于数据集本身还存在这一定的问题，部分图像还存在着多个病灶，被正常得分辨了出来，因此，需要对产出的分割图像进行填洞处理，具体来说就是将最大面积的病灶保留下来，然后讲其他的病灶区域涂黑。

最终，单个模型在官网的100张图片的验证集上取得了jaccard取得了 0.745 的成绩， 将多个表现良好的模型的分割概率结果叠加起来，求平均值，在验证集上取得了 0.765 的成绩，然而，最终在官方的测试集1000张图像中仅仅有 0.728 的成绩。这非常明确地告诉我们，我们的结果在 validation 集上过拟合了，所以在 test 中一下子就被打回了原型。想想 0.728 是我们单个模型在自己的验证集上的一般成绩。

### 五、 改进方向

1. 将 U-Net 修改成 multi-scale 的模式，让网络同时兼顾不同大小的病灶区域
2. 尝试更多的模型

### 六、总结分析

整个比赛的过程中，虽然也学习到了很多细节上的东西，但同时也确实暴露了很多自身的问题和缺陷   
1. 犯的最大的错误是拿着锤子找钉子，而不肯勇敢地放下自己手里的锤子   
    由于自己之前在调试U-net这一个模型的时候，就耗费了不少的时间和精力，所以，在它取得看似还说的过去的结果的时候，就一直在调试这个模型，而没有真正地尝试其他的网络结构。  
2. 知识面太窄了    
    一直在使用tensorflow，就没有转pytorch，导致到真正要用的时候，没能快速地切换；另一方面，是自己阅读的论文，和尝试和体验的网络太少了，这些都需要自己好好反思和跟进的